In [1]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
import PyPDF2




import csv
import json
import pandas as pd


import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

import pytesseract

# Modulos da solucao
import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron



#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


####Config Processamento Pipeline

# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 7. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 8. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"



# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

#Modelo atual
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

'pt_BR.utf8'

In [5]:
#tratamento de exceções
tratamento_excecoes_path = "pipeline_extracao_documentos/3_tratamento_excecoes_batches"

#### paths de objetos para criacao/gestao (dicionarios/datasets)
cnae_dict_path = "pipeline_extracao_documentos/6_geral_administacao/datasets/CNAE_X_ITEM_SERVICO_PREFEITURAS.xlsx"

In [2]:
import modules.extrai_pdf_pesquisavel as Extc

import os
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path

import pytesseract

poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

In [3]:
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v9.xlsx"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"


#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [6]:
# Leitura do arquivo de processamento de notas fiscais
path_to_processamento_nf = "pipeline_extracao_documentos/6_geral_administacao/exports/processamento_nf.xlsx"

df_extract = pd.read_excel(path_to_processamento_nf)

In [7]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_analise_pipe_path = "Em_HOLD/df_mapeamento_e_analise2.xlsx"


#Le a planilha e cria df_documento_recebido
df_analise_pipe = pd.read_excel(df_analise_pipe_path)

# Ajusta o indice
df_analise_pipe.set_index('document_unique_id', inplace=True)

In [8]:
def pesquisa_dict_prefeitura_modelo(texto):

    # 7. ZZZ Dicionário para mapear Prefeitura e/ou CNPJ para um template específico
    templates = {
        ("PREFEITURA DA CIDADE MAGE", "30.693.231/0001-99"): "MAGE_MAICON",
        ("PREFEITURA DA CIDADE MAGE", "23.317.112/0001-76"): "MAGE_MFF",
        ("PREFEITURA DA CIDADE MAGE", None): "MAGE",
        ("PREFEITURA MUNICIPAL DE MAGE", None): "MAGE",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "47.945.459/0001-21"): "SAO_PEDRO_GOAT",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "68.687.722/0001-08"): "SAO_PEDRO_GM",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "34.230.979/0038-06"): "SAO_PEDRO_SUPERMIX",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", None): "SAO_PEDRO",
        ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
        # ... adicione outras combinações aqui
    }




    # 8. ZZZ Consistencia para buscar modelo
    prefeitura_encontrada = None
    cnpj_encontrado = None

    # Verifique cada linha do texto
    for linha in texto:
        for pref, cnpj in templates.keys():
            if pref in linha:
                #print(linha)
                prefeitura_encontrada = pref
            if cnpj and cnpj in linha:
                cnpj_encontrado = cnpj

    # Saímos do loop, agora vamos verificar qual template usar
    if prefeitura_encontrada:
        template_usar = templates.get((prefeitura_encontrada, cnpj_encontrado))
        if not template_usar:
            template_usar = templates.get((prefeitura_encontrada, None), "NAO_PROCESSAR")
    else:
        template_usar = "NAO_PROCESSAR"

    return template_usar, prefeitura_encontrada, cnpj_encontrado





In [9]:
# XXX Usando na criacao da imagem 
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 

# XXX Funcao ajustada para convertere e resize
def convertResize(doc2convert, document_path, image_resized_path):
    
    
    name_image = conv_filename_no_ext(doc2convert)
    
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    #print(f'image_resized_name: {image_resized_name}\n')
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
        image_2work = resized_pages[0]
        
    return image_2work, image_resized_name

In [ ]:
print(f'3 em executa model_frame - antes do for {model} se: {secao} |f_frame_name: {f_frame_name} | f_tipo: {f_tipo}')
    
    

        #print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
        
    return texto_extraido_pil

### Funçoes REGEX e Extracao

In [10]:
# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None
    
    
# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)



#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None
    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        
#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  


    
def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):

    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz           
    #print(resultado_extraido)
    return resultado_extraido

In [11]:
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por')
    return texto_extraido 


# 6. XXX Ajusta texto para PDF RASTER NO CABECALHO
def texto_extraido_cabecalho(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    text_splited = [s.replace(")", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

In [12]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status):
    
    linhas_analise = []
    bloco_1_list = []
    bloco_2_list = []
    bloco_3_list = []
    imagens_list = []  
    new_data = [] 
    
    pre_processo = ['nro_nota', 'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    time_now = cron.timenow_pt_BR()
    
    fase_processo_atual = fase
    atividade_processo_atual = atividade
    status_documento_atual = status
    
    i = 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        
        # 1. Mapeamento de informacoes do DF
        document_unique_id = idx
        seq_df = row['seq']
        batch_name = row['batch']
        fase_processo = row['fase_processo']
        nome_atividade = row['nome_atividade']
        status_documento = row['status_documento']
        original_file_name = row['original_file_name']
        file_directory = row['directory']
        level = row['level']
        d_type = row['level']
        document_type = row['document_type']
        pdf_pesquisavel = row['pdf_pesquisavel']
        one_page_doc = row['one_page']
        modelo = row['modelo']
        
        prefeitura = row['prefeitura']
        
        parent_document_unique_id = row['parent_document_unique_id']
        file_path = row['file_path']
        file_hash = row['file_hash']
        
        # 2. Busca modelo
        if (not status_documento == 'NAO_PROCESSAR') or (not document_type == 'outros') or (not one_page_doc == True):
            
            if atividade_processo_atual == 'extracao_prestador':
            #print(f' 1 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel}\n')
                if status_documento == status_documento_atual:
         
                    print(f'seq_df: {seq_df} status_documento: {status_documento} | modelo: {modelo:>20} | file: {original_file_name:>40} | prefeitura: {prefeitura:>55} | {pdf_pesquisavel}\n')
                    
                    result_list = []
                    
                    dfcnpj_prestador = {}
                    dfincricao_prestador = {}
                    dfdados_prestador = {}
                        
                    # doc2convert = original_file_name
                    section = "2. PRESTADOR DE SERVIÇO"
                    modelo = 'SAO_PEDRO_SUPERMIX'
                    f_tipo = 'frame'
                    
                    dfcnpj_prestador, dfincricao_prestador, dfdados_prestador, textoextraido  = processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo)
                    
                    print(f'\n {dfcnpj_prestador}\n{dfincricao_prestador}\n{dfdados_prestador}\n')
                    
               
      


                i += 1
              
    return textoextraido

In [39]:
# 3B. XXX Efetuo a extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'extracao_prestador' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'OK'

meu_texto_cnpj_prestador = {}

dados_lista_texto_extraido = []
#raw_texto = extracao_pipeline(df_analise_pipe, fase, atividade, status)
textoextraido_nao = extracao_pipeline(subset_df, fase, atividade, status)

seq_df: 20 status_documento: OK | modelo:                 MAGE | file:                              NF 6228.pdf | prefeitura:                            PREFEITURA MUNICIPAL DE MAGE | 1.0

doc pdf_pesquisavel:         2_frame_cnpj_prestador | id: 245 | x0:    0.0 y0:  100.0 x1:  600.0 y1:  236.0 
doc pdf_pesquisavel:    2_frame_inscricao_prestador | id: 249 | x0:    0.0 y0:  100.0 x1:  600.0 y1:  236.0 
doc pdf_pesquisavel:        2_frame_dados_prestador | id: 252 | x0:    0.0 y0:  100.0 x1:  600.0 y1:  236.0 

 {}
{}
{}

seq_df: 60 status_documento: OK | modelo:   SAO_PEDRO_SUPERMIX | file:        1_5145819163654095684_page_24.pdf | prefeitura:             PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA | 0.0

doc raster_pdf:         2_frame_cnpj_prestador | id: 245 | x0:  100.0 y0:  423.0 x1:  550.0 y1:  600.0 
2 extract_fields_cnpj:  CPF/CNPJ:
34.230.979/0038-06

Telefone:
2621 1551


doc raster_pdf:    2_frame_inscricao_prestador | id: 249 | x0:  550.0 y0:  423.0 x1:  929.0 y1:  588.0 

In [38]:
def processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo):
    
    data_box_valores = {'secao': section}
    #lista_texto_extraido = []
    
    nf_cnpj_prestador = {}
    nf_incricao_prestador = {}
    nf_dados_prestador = {}
    
    model = row['modelo']
    
    if pdf_pesquisavel:
        f_tipo = 'frame'
        
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    if pdf_pesquisavel:
        pdf_document = fitz.open(file_path)
        page_number = 0
        page = pdf_document[page_number]
    else:
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        label = row_frame['label']
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        print(f'doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        if pdf_pesquisavel:
            texto_extraido = page.get_text("text", clip=(x0, y0, x1, y1))
            #data_box_valores['processo'] = "PDF_PESQUISAVEL"
            #texto_to_process = texto_pdf
        else:
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            if label == '2_frame_cnpj_prestador':
                nf_cnpj_prestador = extract_fields_cnpj(texto_extraido, original_file_name)
            elif label == '2_frame_inscricao_prestador':
                nf_incricao_prestador = extract_fields_inscricao(texto_extraido, original_file_name)
            elif label == '2_frame_dados_prestador':
                nf_dados_prestador = extract_fields_dados(texto_extraido, original_file_name) 
            #data_box_valores['processo'] = "RASTER_PDF"
            #texto_to_process = texto_extraido_pil

    
    if pdf_pesquisavel:
        pdf_document.close()
        
    return nf_cnpj_prestador, nf_incricao_prestador, nf_dados_prestador, texto_extraido


In [37]:
def extract_fields_cnpj(texto_extraido, original_file_name): 
  
    message_erro = []
    
    data_nf_cnpj_prestador = {}
      
    print(f'2 extract_fields_cnpj:  {texto_extraido}')
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in texto_extraido:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
        if cpf_cnpj_formatado_match:
            data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
            data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
    else:
        data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = None
        data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = None            
            
            
    telefone_str = None

    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        data_nf_cnpj_prestador['telefone'] = telefone_str
    else:
        data_nf_cnpj_prestador['telefone'] = None 
    
    #print(data_nf_cnpj_prestador)    

             
    return data_nf_cnpj_prestador

In [ ]:
for value in dados_lista_texto_extraido[1]:
    #print(f"Chave: {key}")
    #print(f"Valor: {value}")

        print(f"Valor: {value}")
    
    #print()  # Isso imprime uma linha vazia para separar as entradas

In [ ]:
for key, value in dadinho.items():
    #print(f"Chave: {key}")
    #print(f"Valor: {value}")
    if key == '2_frame_cnpj_prestador':
        print(f"kye: {key}, Valor: {value}")
    elif key == '2_frame_inscricao_prestador':
        print(f"kye: {key}, Valor: {value}")
    elif key == '2_frame_dados_prestador':
        print(f"kye: {key}, Valor: {value}")
            

    

In [ ]:
def processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo):
    
    data_box_valores = {'secao': section, 'erros': []}
    lista_texto_extraido = []
    
    if pdf_pesquisavel:
        f_tipo = 'frame'
        
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    if pdf_pesquisavel:
        pdf_document = fitz.open(file_path)
        page_number = 0
        page = pdf_document[page_number]
    else:
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        frame_label = row_frame['label']
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        print(f'doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {frame_label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        if pdf_pesquisavel:
            texto_pdf = page.get_text("text", clip=(x0, y0, x1, y1))
            data_box_valores['processo'] = "PDF_PESQUISAVEL"
            texto_to_process = texto_pdf
        else:
            texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            data_box_valores['processo'] = "RASTER_PDF"
            texto_to_process = texto_extraido_pil
        
        label = row_frame['label']
        if label in ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']:
            #print(f'{data_box_valores["processo"]} {label}: {texto_to_process}')
            extraction_function = globals()[f'extract_fields_{label.split("_")[2]}']
            data_box_valores[label], new_errors = extraction_function(texto_to_process, original_file_name)
            data_box_valores['erros'].extend(new_errors)
            
    lista_texto_extraido.append(data_box_valores)
    
    if pdf_pesquisavel:
        pdf_document.close()
        
    return lista_texto_extraido

In [ ]:
def processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo):
    
    data_box_valores = {'secao': section}
    lista_texto_extraido = []
    
    if pdf_pesquisavel:
        f_tipo = 'frame'
        
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    if pdf_pesquisavel:
        pdf_document = fitz.open(file_path)
        page_number = 0
        page = pdf_document[page_number]
    else:
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        frame_label = row_frame['label']
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        print(f'doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {frame_label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        if pdf_pesquisavel:
            texto_pdf = page.get_text("text", clip=(x0, y0, x1, y1))
            data_box_valores['processo'] = "PDF_PESQUISAVEL"
            texto_to_process = texto_pdf
        else:
            texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            data_box_valores['processo'] = "RASTER_PDF"
            texto_to_process = texto_extraido_pil
        
        label = row_frame['label']
        if label in ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']:
            print(f'{data_box_valores["processo"]} {label}: {texto_to_process}')
            extraction_function = globals()[f'extract_fields_{label.split("_")[2]}']
            data_box_valores[label] = extraction_function(texto_to_process, original_file_name)
            #data_box_valores[label], new_errors = extraction_function(texto_to_process, original_file_name)
            #data_box_valores['erros'].extend(new_errors)
            
    lista_texto_extraido.append(data_box_valores)
    
    if pdf_pesquisavel:
        pdf_document.close()
        
    return lista_texto_extraido


In [ ]:
def processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo):
    
    data_box_valores = {}
    data_box_valores['secao'] = section
    lista_texto_extraido = []
    
    
    if pdf_pesquisavel == True:
        f_tipo = 'frame'
        
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        frame_label = row_frame['label']
    
        if pdf_pesquisavel == True:
            x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
            print(f'doc pdf_pesquisavel: {frame_label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')        

            pdf_document = fitz.open(file_path)
            page_number = 0  # Defina o número da página que deseja analisar
            page = pdf_document[page_number]
            nf_data_prestador = {}
            # Extrair texto dentro do retângulo
            texto_pdf = page.get_text("text", clip=(x0, y0, x1, y1))
            data_box_valores['processo'] = "PDF_PESQUISAVEL"
            label = row_frame['label']
            
            pdf_document.close()
            
            if label == '2_frame_cnpj_prestador':
                print(f'2_frame_cnpj_prestador: {texto_pdf}')
                dados_cnpj_prestador, data_box_valores['erros']  = extract_fields_cnpj_prestador(texto_pdf, original_file_name)
                if dados_cnpj_prestador:
                    data_box_valores[label] = dados_cnpj_prestador
                data_box_valores[label] = dados_cnpj_prestador
                
            
            elif label == '2_frame_inscricao_prestador':
                print(f'2_frame_inscricao_prestador: {texto_pdf}')
                dados_inscricao_prestador, data_box_valores['erros'] = extract_fields_inscricao_prestador(texto_pdf, original_file_name)
                data_box_valores[label] = dados_inscricao_prestador
            
            elif label == '2_frame_dados_prestador':
                print(f'2_frame_dados_prestador: {texto_pdf}')
                data_dados_prestador, data_box_valores['erros'] = extract_fields_dados_prestador(texto_pdf, original_file_name)
                data_box_valores[label] = data_dados_prestador      
            
        else:
            
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            print(f'doc raster_pdf: {frame_label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
            image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

            texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            data_box_valores['processo'] = "RASTER_PDF"
            #lista_texto_extraido.append(texto_extraido_pil)
            label = row_frame['label']
            
            if label == '2_frame_cnpj_prestador':
                print(f'RASTER 2_frame_cnpj_prestador: {texto_extraido_pil}')
                dados_cnpj_prestador, data_box_valores['erros'] = extract_fields_cnpj_prestador(texto_extraido_pil, original_file_name)
                data_box_valores[label] = dados_cnpj_prestador
                
                
            elif label == '2_frame_inscricao_prestador':
                print(f'RASTER 2_frame_inscricao_prestador: {texto_extraido_pil}')
                dados_inscricao_prestador, data_box_valores['erros'] = extract_fields_inscricao_prestador(texto_extraido_pil, original_file_name)
                data_box_valores[label] = dados_inscricao_prestador
               
                
            elif label == '2_frame_dados_prestador':
                print(f'RASTER 2_frame_dados_prestador: {texto_extraido_pil}')
                data_dados_prestador, data_box_valores['erros'] = extract_fields_dados_prestador(texto_extraido_pil, original_file_name)
                data_box_valores[label] = data_dados_prestador
                

    lista_texto_extraido.append(data_box_valores)
        
        
    #print(f'frame_label: {frame_label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
    return lista_texto_extraido, 

In [ ]:
def extract_processar_dados_antigo(row, file_path, image_2work, section, modelo, f_tipo):
    
    data_box_valores = {}
    data_box_valores['secao'] = section
    lista_texto_extraido = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        
        texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1))
        lista_texto_extraido.append(texto_extraido_pil)
        
        frame_id = row_frame['id']
        frame_label = row_frame['label']
        
        print(f'frame_label: {frame_label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        

        label = row_frame['label']
        data_box_valores[label] = texto_extraido_pil
        
        
    return data_box_valores    
    

In [ ]:
def extract_fields_cnpj_prestador_raster_pdf(row, texto_cnpj_prestador): 
  
    message_erro = []
  
    # # print(f'2 dentro do try {model} se: {secao} |f_frame_name: {f_frame_name} | f_tipo: {f_tipo} | raw_texto_pil: {raw_texto_pil}')
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in texto_cnpj_prestador:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_cnpj_prestador)
        if cpf_cnpj_formatado_match:
            cpf_cnpj_com_mascara_prestador = cpf_cnpj_formatado_match.group(1)
            cpf_cnpj_sem_mascara_prestador = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
    else:
        cpf_cnpj_com_mascara_prestador = None
        cpf_cnpj_sem_mascara_prestador = None            
            
            
    telefone_str = None

    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_cnpj_prestador)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        telefone_prestatador_frame = telefone_str
    else:
        telefone_prestatador_frame = None 

             
    return cpf_cnpj_com_mascara_prestador, cpf_cnpj_sem_mascara_prestador, telefone_prestatador_frame

In [11]:
valores = [20, 60]
subset_df = df_analise_pipe[df_analise_pipe['seq'].isin(valores)]

In [ ]:
subset_df

In [ ]:
df_analise_pipe.query('status_documento == "OK" and pdf_pesquisavel == False ')

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 63')

In [ ]:
df_analise_pipe

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['label'] == f_frame_name) & (frames_nf_v4_df['type'] == f_tipo)]

for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    
    x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
    texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1)) # PONTO IMPORTANTE
    print(f'3 - executa: {model} | {secao} | {f_frame_name}  x0: {x0} y0: {y0} x1: {x1} y1: {y1}')
    


In [ ]:
secao = "1 - CABECALHO"
section = "1 - CABECALHO"
f_frame_name = "1_frame_dados_nf"


In [ ]:
frames_nf_v4_df

In [ ]:
section = "2. PRESTADOR DE SERVIÇO"
f_tipo = "sframe_field"
modelo = "SAO_PEDRO_SUPERMIX"

f_frame_name = "1_frame_dados_nf"

In [ ]:
section = "2. PRESTADOR DE SERVIÇO"
model = 'SAO_PEDRO_SUPERMIX'
f_tipo = 'frame'
f_frame_name = "2_frame_inscricao_prestador"

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['label'] == f_frame_name) & (frames_nf_v4_df['type'] == f_tipo)]

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == modelo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]

In [ ]:
for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    
    x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
    
    texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1))
    
    frame_id = row_frame['id']
    frame_label = row_frame['label']
    
    print(f'id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} | frame_label: {frame_label}')

In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas/1_5145819163654095684_page_31.jpg"

In [ ]:
# Define a mapping for color names to RGB values
color_mapping = {
    "red": (255, 0, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "green": (0, 128, 50),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0)
}

# Reload the image to start fresh
image = Image.open(name_image_2work)
draw = ImageDraw.Draw(image)

# Define a font size for the labels using the default PIL font
font_size = 100
#font = ImageFont.load_default()

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf", 30, encoding="unic")

# Update the draw_box function to use the larger font size with the default font
def draw_box(row):
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    color = color_mapping.get(row['color'], (0, 0, 0)) # Default to black if color not found
    draw.rectangle([x0, y0, x1, y1], outline=color, width=3)
    label = str(row['label']) if pd.notnull(row['label']) else None # Check for missing label
    if label:
        draw.text((x0 + 5, y0 + 5), label, fill=color, font=font)

# Draw the boundaries
#draw_box(boundaries_info)


def draw_box_model(modelo,
                   boundaries_info=None,
                   sections_info=None,
                   frames_info=None,
                   field_boxes_info=None,
                   draw_boundaries=True,
                   draw_sections=True,
                   draw_frames=True,
                   draw_field_boxes=True):
    
    # Draw boundaries if requested
    if draw_boundaries and boundaries_info is not None:
        filtered_boundaries_info = boundaries_info[boundaries_info['model'] == modelo]
        for index, row in filtered_boundaries_info.iterrows():
            draw_box(row)

    # Draw sections if requested
    if draw_sections and sections_info is not None:
        filtered_sections_info = sections_info[sections_info['model'] == modelo]
        for index, row in filtered_sections_info.iterrows():
            draw_box(row)
            
    # Draw frames if requested
    if draw_frames and frames_info is not None:
        filtered_frames_info = frames_info[frames_info['model'] == modelo]
        for index, row in filtered_frames_info.iterrows():
            draw_box(row)
            
    # Draw field boxes if requested
    if draw_field_boxes and field_boxes_info is not None:
        filtered_field_boxes_info = field_boxes_info[field_boxes_info['model'] == modelo]
        for index, row in filtered_field_boxes_info.iterrows():
            draw_box(row)
    
    # Show the image with selected drawings
    image.show()

In [ ]:
modelo = 'SAO_PEDRO_SUPERMIX'

In [ ]:
# To draw everything
draw_box_model(modelo, boundaries_info, sections_info, frames_info, field_boxes_info)


In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


In [ ]:
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_type = row_frame['type']
    frame_section = row_frame['section_json']
    frame_reference = row_frame['reference']
    frame_father = row_frame['father']
    frame_id = row_frame['id']

In [ ]:
frames_nf_v4_filter = filtrar_df_multiplos(frames_nf_v4_df, section_json="== '2. PRESTADOR DE SERVIÇO'", model="== 'SAO_PEDRO_SUPERMIX'", type="== 'frame'")

In [ ]:
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

In [ ]:
def filtrar_df_multiplos(df, **kwargs):
    query = " & ".join(f"{key} {value}" if isinstance(value, str) and value.startswith((">", "<", ">=", "<=", "!=", "==")) else f"{key} == @value" for key, value in kwargs.items())
    result = df.query(query, local_dict=kwargs)
    return result


In [ ]:
frames_nf_v4_filter

In [ ]:
for idx, row in frames_nf_v4_filter.iterrows:
    print(idx, row)

In [ ]:
x0 = result['x0'].values[0]
x1 = result['x1'].values[0]
y0 = result['y0'].values[0]
y1 = result['y1'].values[0]
label = result['label'].values[0]

print(f' x0: {x0} y0: {y0} x1: {x1} y1: {y1} label: {label}')

In [ ]:
def get_coordenates_tipo(section, modelo, tipo):
    
    result = filtrar_df(frames_nf_v4_df, section_json=section, model=modelo, type=tipo)
    x0 = result['x0'].values[0]
    x1 = result['x1'].values[0]
    y0 = result['y0'].values[0]
    y1 = result['y1'].values[0]
    label = result['label'].values[0]
    
    
    return x0, x1, y0, y1, label

In [ ]:
get_coordenates_tipo(section, modelo, tipo)

In [ ]:
def get_coordenates(section, modelo):
    
    result = filtrar_df(frames_nf_v4_df, section_json=section, model=modelo)
    x0 = result['x0'].values[0]
    x1 = result['x1'].values[0]
    y0 = result['y0'].values[0]
    y1 = result['y1'].values[0]
    label = result['label'].values[0]
    
    
    return x0, x1, y0, y1, label

In [ ]:
get_coordenates(section, 'SAO_PEDRO_SUPERMIX')